In [2]:
import os

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from pytorch_lightning.core.lightning import LightningModule

class LitModel(LightningModule):

    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def train_dataloader(self):
        dataset = MNIST(os.getcwd(), train=True, download=True, transform=transforms.ToTensor())
        loader = DataLoader(dataset, batch_size=32, num_workers=4, shuffle=True)
        return loader

In [5]:
from pytorch_lightning import Trainer

model = LitModel()

# most basic trainer, uses good defaults
trainer = Trainer(gpus=1, num_nodes=1)
trainer.fit(model)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
----------------------------
0 | l1   | Linear | 7 K   


Extracting /data/projects/research-first/lighting/MNIST/raw/train-images-idx3-ubyte.gz to /data/projects/research-first/lighting/MNIST/raw


Extracting /data/projects/research-first/lighting/MNIST/raw/train-labels-idx1-ubyte.gz to /data/projects/research-first/lighting/MNIST/raw


Extracting /data/projects/research-first/lighting/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/projects/research-first/lighting/MNIST/raw


Extracting /data/projects/research-first/lighting/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/projects/research-first/lighting/MNIST/raw
Processing...
Done!





/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


/home/curt/.pyenv/versions/3.8.3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Did not find hyperparameters at model hparams. Saving checkpoint without hyperparameters.
  warnings.warn(*args, **kwargs)
Detected KeyboardInterrupt, attempting graceful shutdown...


1